In [2]:
import pandas as pd
import sys
from preprocess import train_val_test_split, prepare_splitted_data

In [3]:
PROJECT_PATH = '~/Projects/negative_feedback/'
DATA_PATH = f'{PROJECT_PATH}/data/ml-1m/'

RELEVANCE_COL = 'rating'
RELEVANCE_THRESHOLD = 4

In [4]:
path_to_df = '~/Projects/negative_feedback/data/ratings.dat'
data = pd.read_csv(path_to_df, sep='::', engine='python', names=['user_id','item_id','rating','timestamp'])

In [5]:
data.duplicated(['user_id', 'item_id']).sum()

0

In [6]:
(data.groupby('item_id').count()['user_id'] < 5).mean()

0.07825148407987048

In [7]:
(data.groupby('user_id').count()['item_id'] < 5).mean()

0.0

In [8]:
data.groupby('user_id').count()['item_id'].median()

96.0

In [20]:
train, val, test = train_val_test_split(data, RELEVANCE_THRESHOLD, RELEVANCE_COL, test_quantile=0.9)

Filtering items..
Number of items before 3706
Number of items after 3416
Interactions length before: 1000209
Interactions length after: 999611
Filtering users..
Number of users before 6040
Number of users after 6040
Interactions length before: 999611
Interactions length after: 999611


In [18]:
val.user_id.nunique()

589

In [19]:
train.user_id.nunique()

6011

In [20]:
def filter_last_positive(df):

    df = df.sort_values(['user_id', 'timestamp'])
    last_item = df.groupby('user_id').tail(1)
    last_item = last_item[last_item[RELEVANCE_COL] >= RELEVANCE_THRESHOLD]
    df = df[df['user_id'].isin(last_item['user_id'])]

    return df

In [21]:
val = filter_last_positive(val)

In [22]:
val.user_id.nunique()

134

In [23]:
train.to_parquet(f'{DATA_PATH}/train.parquet', index=False)
test.to_parquet(f'{DATA_PATH}/test.parquet', index=False)
val.to_parquet(f'{DATA_PATH}/validation.parquet', index=False)

In [22]:
data = pd.concat([train, val, test])
data.groupby('user_id')['item_id'].count().mean()

165.5874230577275

In [29]:
data.groupby('user_id')['item_id'].count().std()

192.8330370240466

In [23]:
data.item_id.nunique()

3412

In [28]:
np.mean(data.rating < 4)

0.424631233761928